**API UTILIZADA : API de eBird 2.0**

eBird es un proyecto de ciencia ciudadana que recopila datos sobre aves de todo el mundo. La base de datos de eBird incluye más de 100 millones de registros que son de libre acceso y se utilizan para la investigación, la conservación y la educación.
Los datos de eBird se utilizan para:
\
-Investigar y monitorear las aves\
-Planificar la conservación\
-Manejar áreas y hábitats\
-Evaluar poblaciones de especies\
-Proteger hábitats\
-Elaborar leyes y políticas

Los datos de eBird son un recurso poderoso para la ciencia y la conservación.

eBird Argentina se nutre del trabajo voluntario de decenas de personas, quienes día a día se encargan de la curación y revisión de datos, mantienen actualizados canales de comunicación, y responden a las inquietudes de los usuarios y usuarias de la plataforma. Su trabajo es fundamental para garantizar el correcto funcionamiento del sistema, y para que eBird refleje datos de alta calidad, que puedan ser aprovechados por toda persona apasionada por la observación de aves, por educadores/as, por agencias gubernamentales, ONGs, y por la comunidad académica.

In [ ]:
!pip install requests
!pip install deltalake
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 42.0 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import pyarrow as pa
from deltalake import write_deltalake, DeltaTable
from deltalake.exceptions import TableNotFoundError
from datetime import datetime, timedelta

In [ ]:
from configparser import ConfigParser

In [ ]:
import os
print(os.listdir('/content/sample_data'))

['anscombe.json', 'README.md', 'pipeline.conf', 'mnist_test.csv', 'california_housing_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv']


In [ ]:
parser = ConfigParser()

parser.read("/content/sample_data/pipeline.conf")

['/content/sample_data/pipeline.conf']

In [ ]:
api_credentials = parser["ebird_accesoApi"]

In [ ]:
api_credentials["ebird_token"]

'17hkrgb0cps4'

In [ ]:
ebird_token = api_credentials["ebird_token"]

In [ ]:
headers = {
    'X-eBirdApiToken': ebird_token,
}

In [ ]:
#para casos de extracciones incrementrales
Ubicacion = parser["pais_región"]
lugar = Ubicacion["lugar"]

In [ ]:
params = {}

In [ ]:

def get_data(url, params=None, headers=None):
    """
    Realiza una solicitud GET a una API para obtener datos.

    Parámetros:
    url (str): La URL del endpoint
    params (dict): Parámetros de consulta para enviar con la solicitud.
    data_field (str): El nombre del campo en el JSON que contiene los datos.
    headers (dict): Encabezados para enviar con la solicitud.

    Retorna:
    dict: Los datos obtenidos de la API en formato JSON.
    """
    try:
        response = requests.get(url, params=params, headers=headers)
        print(f"El codigo de respuesta es : {response.status_code}")
        response.raise_for_status()  # Levanta una excepción si hay un error en la respuesta HTTP.

        # Verificar si los datos están en formato JSON.
        try:
            data = response.json()
            type(data)

        except:
            print("El formato de respuesta no es el esperado")
            return None
        return data

    except requests.exceptions.RequestException as e:
        # Capturar cualquier error de solicitud, como errores HTTP.
        print(f"La petición ha fallado. Código de error : {e}")
        return None

def build_table(json_data):
    """
    Construye un DataFrame de pandas a partir de datos en formato JSON.

    Parámetros:
    json_data (dict): Los datos en formato JSON obtenidos de una API.

    Retorna:
    DataFrame: Un DataFrame de pandas que contiene los datos.
    """
    try:
        df = pd.json_normalize(json_data)
        return df
    except:
        print("Los datos no están en el formato esperado")
        return None

def save_data_as_delta(df, path, mode="overwrite", partition_cols=None):
    """
    Guarda un dataframe en formato Delta Lake en la ruta especificada.
    A su vez, es capaz de particionar el dataframe por una o varias columnas.
    Por defecto, el modo de guardado es "overwrite".

    Args:
      df (pd.DataFrame): El dataframe a guardar.
      path (str): La ruta donde se guardará el dataframe en formato Delta Lake.
      mode (str): El modo de guardado. Son los modos que soporta la libreria
      deltalake: "overwrite", "append", "error", "ignore".
      partition_cols (list or str): La/s columna/s por las que se particionará el
      dataframe. Si no se especifica, no se particionará.
    """
    write_deltalake(
        path, df, mode=mode, partition_by=partition_cols
    )

def save_new_data_as_delta(new_data, data_path, predicate, partition_cols=None):
    """
    Guarda solo nuevos datos en formato Delta Lake usando la operación MERGE,
    comparando los datos ya cargados con los datos que se desean almacenar
    asegurando que no se guarden registros duplicados.

    Args:
      new_data (pd.DataFrame): Los datos que se desean guardar.
      data_path (str): La ruta donde se guardará el dataframe en formato Delta Lake.
      predicate (str): La condición de predicado para la operación MERGE.
    """

    try:
      dt = DeltaTable(data_path)
      new_data_pa = pa.Table.from_pandas(new_data)
      # Se insertan en target, datos de source que no existen en target
      dt.merge(
          source=new_data_pa,
          source_alias="source",
          target_alias="target",
          predicate=predicate
      ) \
      .when_not_matched_insert_all() \
      .execute()

    # Si no existe la tabla Delta Lake, se guarda como nueva
    except TableNotFoundError:
      save_data_as_delta(new_data, data_path, partition_cols=partition_cols)

def upsert_data_as_delta(data, data_path, predicate, partition_cols=None):
    """
    Guardar datos en formato Delta Lake usando la operacion MERGE.
    Cuando no haya registros coincidentes, se insertarán nuevos registros.
    Cuando haya registros coincidentes, se actualizarán los campos.

    Args:
      data (pd.DataFrame): Los datos que se desean guardar.
      data_path (str): La ruta donde se guardará el dataframe en formato Delta Lake.
      predicate (str): La condición de predicado para la operación MERGE.
    """
    try:
        dt = DeltaTable(data_path)
        data_pa = pa.Table.from_pandas(data)
        dt.merge(
            source=data_pa,
            source_alias="source",
            target_alias="target",
            predicate=predicate
        ) \
        .when_matched_update_all() \
        .when_not_matched_insert_all() \
        .execute()
    except TableNotFoundError:
        save_data_as_delta(data, data_path, partition_cols=partition_cols)


def conv_to_csv(dt, Nombre):
    # Si se desea tener la info en formato CSV
    # Convertir a DataFrame de pandas
    dx= dt.to_pandas()
    #fecha_actual = datetime.now().strftime("%d-%m-%Y")
    datos = Nombre + '.csv'
    print(datos)
    dx.to_csv(datos, index=False)


**EXTRACCION FULL**

**Endpoint: eBird Taxonomy** ("https://api.ebird.org/v2/ref/taxonomy/ebird")

**Publicación de nuevas versiones en eBird:** Cada año, eBird lanza una nueva versión de su taxonomía, con el objetivo de reflejar los avances científicos y cambios en la comprensión de la evolución y clasificación de las aves. La taxonomía actualizada incluye no solo nuevas especies que han sido descubiertas o reconocidas oficialmente, sino también cambios en las clasificaciones, tales como divisiones de especies existentes (especiación) o fusiones de especies que antes se consideraban separadas (lumping). La nueva versión se anuncia previamente en el sitio web de eBird y generalmente se incluye una lista detallada de los cambios implementados.

Ante esto se decide usar el append y realizar una carga completa desde cero una vez al año(cuando se anuncie la nueva versión de taxonomia), se decide utilizar un directorio incluyendo en su denominación el año correspondiente a la versión.



In [ ]:
url = 'https://api.ebird.org/v2/ref/taxonomy/ebird'

In [ ]:
#puede devolver datos en formato csv y json pero si no se indica lo contrario, devuelve formato csv
params = { 'fmt' : 'json' }

In [ ]:
#Acceso a endpoint y extraigo datos en formato json
json_data = get_data(url, params, headers)

El codigo de respuesta es : 200


In [ ]:
#obtención de un DataFrame de pandas a partir de datos
df_Taxon= build_table(json_data)

In [ ]:
df_Taxon.head()

,sciName,comName,speciesCode,category,taxonOrder,bandingCodes,comNameCodes,sciNameCodes,order,familyCode,familyComName,familySciName,reportAs,extinct,extinctYear
0,Struthio camelus,Common Ostrich,ostric2,species,2.0,[],[COOS],[STCA],Struthioniformes,struth1,Ostriches,Struthionidae,NaN,NaN,NaN
1,Struthio molybdophanes,Somali Ostrich,ostric3,species,7.0,[],[SOOS],[STMO],Struthioniformes,struth1,Ostriches,Struthionidae,NaN,NaN,NaN
2,Struthio camelus/molybdophanes,Common/Somali Ostrich,y00934,slash,8.0,[],"[SOOS, COOS]","[STMO, STCA]",Struthioniformes,struth1,Ostriches,Struthionidae,NaN,NaN,NaN
3,Casuarius casuarius,Southern Cassowary,soucas1,species,10.0,[],[SOCA],[CACA],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,NaN,NaN,NaN
4,Casuarius bennetti,Dwarf Cassowary,dwacas1,species,11.0,[],[DWCA],[CABE],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,NaN,NaN,NaN


In [ ]:
#se almacena completa la taxonomia en un directorio correspondiente a la version/año
fecha_actual = datetime.now().strftime("%Y")
print(fecha_actual)
Nombre = "Taxonomia-" + fecha_actual
print(Nombre)
bronze_dir = "datalake/bronze/ebird_api"
taxon_raw_dir = f"{bronze_dir}/taxonomia-{fecha_actual}"
print(taxon_raw_dir)


2024
Taxonomia-2024
datalake/bronze/ebird_api/taxonomia-2024


In [ ]:
#Almacena un dataframe en formato Delta Lake
save_data_as_delta(df_Taxon, taxon_raw_dir, mode="append")

In [ ]:
#A modo de chequeo, leemos los datos guardados y contamos la cantidad de fila
dt = DeltaTable(taxon_raw_dir)
print(f"Cant de filas: {dt.to_pandas().shape[0]}")

Cant de filas: 17415


In [ ]:
# Si se desea tener la info en formato CSV
Nombre = "Taxonomia"
conv_to_csv(dt, Nombre)

Taxonomia
Taxonomia13-11-2024.csv


**TRANSFORMACIÓN:** Comenzamos la transformación sea usando el frame que ya tenemos en memoria o cargandolo desde deltaLake bronze.
En este caso realicé las siguientes operaciones:
 Eliminación o reemplazo de nulos, Conversión de tipos de datos de columnas,- eliminación de columna entera y agregaciones por medio de GROUP BY y funciones count.

In [ ]:
# cargandolo desde deltaLake bronze si es necesario y no lo tenemos ya en memoria
dt = DeltaTable(taxon_raw_dir)

In [ ]:
df = dt.to_pandas()




In [ ]:

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17415 entries, 0 to 17414
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sciName        17415 non-null  object 
 1   comName        17415 non-null  object 
 2   speciesCode    17415 non-null  object 
 3   category       17415 non-null  object 
 4   taxonOrder     17415 non-null  float64
 5   bandingCodes   17415 non-null  object 
 6   comNameCodes   17415 non-null  object 
 7   sciNameCodes   17415 non-null  object 
 8   order          17413 non-null  object 
 9   familyCode     17402 non-null  object 
 10  familyComName  17402 non-null  object 
 11  familySciName  17402 non-null  object 
 12  reportAs       3819 non-null   object 
 13  extinct        175 non-null    object 
 14  extinctYear    175 non-null    float64
dtypes: float64(2), object(13)
memory usage: 16.5 MB


In [ ]:
# Contamos la cantidad de valores nulos en el campo clave
df["speciesCode"].isnull().sum()

0

In [ ]:
df["bandingCodes"].isnull().sum()


0

In [ ]:
df["bandingCodes"]

,bandingCodes
0,[]
1,[]
2,[]
3,[]
4,[]
...,...
17410,[]
17411,[]
17412,[]
17413,[]


In [ ]:
# se da de baja la columna ya que no es un dato necesario
 df = df.drop("bandingCodes", axis=1)

IndentationError: unexpected indent (<ipython-input-32-913ce474b9df>, line 2)

In [ ]:
df.head()

,sciName,comName,speciesCode,category,taxonOrder,comNameCodes,sciNameCodes,order,familyCode,familyComName,familySciName,reportAs,extinct,extinctYear
0,Struthio camelus,Common Ostrich,ostric2,species,2.0,[COOS],[STCA],Struthioniformes,struth1,Ostriches,Struthionidae,None,None,NaN
1,Struthio molybdophanes,Somali Ostrich,ostric3,species,7.0,[SOOS],[STMO],Struthioniformes,struth1,Ostriches,Struthionidae,None,None,NaN
2,Struthio camelus/molybdophanes,Common/Somali Ostrich,y00934,slash,8.0,"[SOOS, COOS]","[STMO, STCA]",Struthioniformes,struth1,Ostriches,Struthionidae,None,None,NaN
3,Casuarius casuarius,Southern Cassowary,soucas1,species,10.0,[SOCA],[CACA],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,None,None,NaN
4,Casuarius bennetti,Dwarf Cassowary,dwacas1,species,11.0,[DWCA],[CABE],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,None,None,NaN


In [ ]:
df["taxonOrder"].isnull().sum()


0

In [ ]:
df["taxonOrder"] = df["taxonOrder"].astype("int16")

In [ ]:
df.head()

,sciName,comName,speciesCode,category,taxonOrder,comNameCodes,sciNameCodes,order,familyCode,familyComName,familySciName,reportAs,extinct,extinctYear
0,Struthio camelus,Common Ostrich,ostric2,species,2,[COOS],[STCA],Struthioniformes,struth1,Ostriches,Struthionidae,None,None,NaN
1,Struthio molybdophanes,Somali Ostrich,ostric3,species,7,[SOOS],[STMO],Struthioniformes,struth1,Ostriches,Struthionidae,None,None,NaN
2,Struthio camelus/molybdophanes,Common/Somali Ostrich,y00934,slash,8,"[SOOS, COOS]","[STMO, STCA]",Struthioniformes,struth1,Ostriches,Struthionidae,None,None,NaN
3,Casuarius casuarius,Southern Cassowary,soucas1,species,10,[SOCA],[CACA],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,None,None,NaN
4,Casuarius bennetti,Dwarf Cassowary,dwacas1,species,11,[DWCA],[CABE],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,None,None,NaN


In [ ]:
df["extinctYear"].notnull().sum()

175

In [ ]:
# Filtra el DataFrame mostrando solo las filas donde no es nulo
df_no_nulos = df[df["extinctYear"].notnull()]

In [ ]:
df_no_nulos



,sciName,comName,speciesCode,category,taxonOrder,comNameCodes,sciNameCodes,order,familyCode,familyComName,familySciName,reportAs,extinct,extinctYear
232,Alopochen mauritiana,Mauritius Shelduck,maushe1,species,406,[MASH],[ALMA],Anseriformes,anatid1,"Ducks, Geese, and Waterfowl",Anatidae,None,True,0.0
233,Alopochen kervazoi,Reunion Shelduck,reushe1,species,407,[RESH],[ALKE],Anseriformes,anatid1,"Ducks, Geese, and Waterfowl",Anatidae,None,True,0.0
241,Tadorna cristata,Crested Shelduck,creshe1,species,415,[CRSH],[TACR],Anseriformes,anatid1,"Ducks, Geese, and Waterfowl",Anatidae,None,True,1916.0
268,Chenonetta finschi,Finsch's Duck,finduc1,species,446,[FIDU],[CHFI],Anseriformes,anatid1,"Ducks, Geese, and Waterfowl",Anatidae,None,True,1760.0
294,Mareca strepera couesi,Gadwall (Coues's),gadwal2,issf,489,[GADW],[MAST],Anseriformes,anatid1,"Ducks, Geese, and Waterfowl",Anatidae,gadwal,True,1874.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15631,Carpodacus ferreorostris,Bonin Grosbeak,bongro1,species,31878,[BOGR],[CAFE],Passeriformes,fringi1,"Finches, Euphonias, and Allies",Fringillidae,None,True,1900.0
15712,Haemorhous mexicanus mcgregori,House Finch (McGregor's),houfin2,issf,32047,[HOFI],[HAME],Passeriformes,fringi1,"Finches, Euphonias, and Allies",Fringillidae,houfin,True,1938.0
16139,Ammospiza maritima nigrescens,Seaside Sparrow (Dusky),dusspa1,issf,-32592,[SESP],[AMMA],Passeriformes,passer3,New World Sparrows,Passerellidae,seaspa,True,1983.0
16210,Pipilo naufragus,Bermuda Towhee,bertow1,species,-32391,[BETO],[PINA],Passeriformes,passer3,New World Sparrows,Passerellidae,None,True,1611.0


In [ ]:
#tiene nulos por eso cancela con error
df["extinctYear"] = df["extinctYear"].astype("int16")

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
# Rellenamos los NaN con 0 y paamos a entero de 16
df['extinctYear'] = df['extinctYear'].fillna(0).astype('int16')


In [ ]:
df

,sciName,comName,speciesCode,category,taxonOrder,comNameCodes,sciNameCodes,order,familyCode,familyComName,familySciName,reportAs,extinct,extinctYear
0,Struthio camelus,Common Ostrich,ostric2,species,2,[COOS],[STCA],Struthioniformes,struth1,Ostriches,Struthionidae,None,None,0
1,Struthio molybdophanes,Somali Ostrich,ostric3,species,7,[SOOS],[STMO],Struthioniformes,struth1,Ostriches,Struthionidae,None,None,0
2,Struthio camelus/molybdophanes,Common/Somali Ostrich,y00934,slash,8,"[SOOS, COOS]","[STMO, STCA]",Struthioniformes,struth1,Ostriches,Struthionidae,None,None,0
3,Casuarius casuarius,Southern Cassowary,soucas1,species,10,[SOCA],[CACA],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,None,None,0
4,Casuarius bennetti,Dwarf Cassowary,dwacas1,species,11,[DWCA],[CABE],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17410,Saltator fuliginosus,Black-throated Grosbeak,bltgro2,species,-29976,[BTGR],[SAFU],Passeriformes,thraup2,Tanagers and Allies,Thraupidae,None,None,0
17411,Saltator sp.,saltator sp.,saltat1,spuh,-29975,[],[],Passeriformes,thraup2,Tanagers and Allies,Thraupidae,None,None,0
17412,Thraupidae sp.,Thraupidae sp.,thraup3,spuh,-29974,[],[],Passeriformes,thraup2,Tanagers and Allies,Thraupidae,None,None,0
17413,Passeriformes sp.,passerine sp.,passer1,spuh,-29973,[],[],Passeriformes,None,None,None,None,None,0


Para recordar: Dado que el tipo object no es tan específico, ocupa más espacio en memoria y tiene una menor eficiencia de almacenamiento y procesamiento en comparación con tipos específicos como bool, int, o float.

In [ ]:
df["extinct"] = df["extinct"].astype("bool")

In [ ]:
df = df = df.drop("reportAs", axis=1)

In [ ]:
df.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17415 entries, 0 to 17414
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   sciName        17415 non-null  object
 1   comName        17415 non-null  object
 2   speciesCode    17415 non-null  object
 3   category       17415 non-null  object
 4   taxonOrder     17415 non-null  int16 
 5   comNameCodes   17415 non-null  object
 6   sciNameCodes   17415 non-null  object
 7   order          17413 non-null  object
 8   familyCode     17402 non-null  object
 9   familyComName  17402 non-null  object
 10  familySciName  17402 non-null  object
 11  extinct        17415 non-null  bool  
 12  extinctYear    17415 non-null  int16 
dtypes: bool(1), int16(2), object(10)
memory usage: 13.4 MB


La columna Order no se toca porque es importante para las taxonomias. **Contiene** un nombre de Orden. Orden es un cayegoría que incluye o agrupa un número de familias y por lo tanto, de especies incluídas en esas familias. Por ejemplo : Los Anseriformes son un orden de aves neognatas que comprende 162 especies1​ repartidas en tres familias: los Anhimidae (con 3 especies), Anseranatidae (con una sola especie), y los Anatidae (con 156 o más especies de ánades o patos; ocas, gansos o ánsares; cisnes; yaguasas; serretas; eideres; barnaclas; cauquenes; porrones; y tarros). El grupo fue bautizado por el nombre latino de la oca «Anser» (la misma etimología que el término español ánsar), y por ello el nombre de este orden significa «los que tienen forma de oca».
comNameCodes se deja este campo a pesar de tener muchos vacios,ya que disponemos del nombre correspondiente a la especie, y seguramente es puntero a alguna tabla que se conecta con los códigos de nombre existentes.

**Finalmente se logra una reducción de memoria de 16.5 MB a memory usage: 13.4 MB**

In [ ]:
df


,sciName,comName,speciesCode,category,taxonOrder,comNameCodes,sciNameCodes,order,familyCode,familyComName,familySciName,extinct,extinctYear
0,Struthio camelus,Common Ostrich,ostric2,species,2,[COOS],[STCA],Struthioniformes,struth1,Ostriches,Struthionidae,False,0
1,Struthio molybdophanes,Somali Ostrich,ostric3,species,7,[SOOS],[STMO],Struthioniformes,struth1,Ostriches,Struthionidae,False,0
2,Struthio camelus/molybdophanes,Common/Somali Ostrich,y00934,slash,8,"[SOOS, COOS]","[STMO, STCA]",Struthioniformes,struth1,Ostriches,Struthionidae,False,0
3,Casuarius casuarius,Southern Cassowary,soucas1,species,10,[SOCA],[CACA],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,False,0
4,Casuarius bennetti,Dwarf Cassowary,dwacas1,species,11,[DWCA],[CABE],Casuariiformes,casuar1,Cassowaries and Emu,Casuariidae,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17410,Saltator fuliginosus,Black-throated Grosbeak,bltgro2,species,-29976,[BTGR],[SAFU],Passeriformes,thraup2,Tanagers and Allies,Thraupidae,False,0
17411,Saltator sp.,saltator sp.,saltat1,spuh,-29975,[],[],Passeriformes,thraup2,Tanagers and Allies,Thraupidae,False,0
17412,Thraupidae sp.,Thraupidae sp.,thraup3,spuh,-29974,[],[],Passeriformes,thraup2,Tanagers and Allies,Thraupidae,False,0
17413,Passeriformes sp.,passerine sp.,passer1,spuh,-29973,[],[],Passeriformes,None,None,None,False,0


**Agrupamos por orden/familia y obtenemos las familias que componen cada Orden y la cantidad de especies que  tiene en total cada familia**


In [ ]:
df.groupby(['order', 'familyCode']
           ).agg(
               {
                'speciesCode': 'count'}
          ).rename(
        columns={
            'speciesCode': 'cantidad-especies'}
          )


cantidad-especies
order           familyCode                   
Accipitriformes accipi1                   420
                pandio1                     5
                sagitt1                     1
Aegotheliformes aegoth1                    13
Anseriformes    anatid1                   460
...                                       ...
Suliformes      fregat1                    10
                phalac1                    61
                sulida1                    26
Tinamiformes    tinami1                    60
Trogoniformes   trogon1                    70

[254 rows x 1 columns]

In [ ]:
#Total de especies para cada orden de la taxonomía
df["order"].value_counts()

,count
order,
Passeriformes,10327
Apodiformes,726
Charadriiformes,697
Piciformes,658
Psittaciformes,590
Columbiformes,481
Anseriformes,464
Galliformes,436
Accipitriformes,426


In [ ]:
#se almacena completa la taxonomia depurada en un directorio Silver correspondiente a la version/año
fecha_actual = datetime.now().strftime("%Y")
Nombre = "Taxonomia-" + fecha_actual
bronze_dir = "datalake/silver/ebird_api"
taxon_raw_dir = f"{bronze_dir}/taxonomia-{fecha_actual}"


In [ ]:
#Almacena un dataframe en formato Delta Lake
save_data_as_delta(df, taxon_raw_dir, mode="append")

In [ ]:
#A modo de chequeo, leemos los datos guardados y contamos la cantidad de fila
dt = DeltaTable(taxon_raw_dir)
print(f"Cant de filas: {dt.to_pandas().shape[0]}")

Cant de filas: 17415


**Escenario inventado para Extracción Full con el mismo endpoint:**Se eligió utilizar Merge con inserción sin update, ya que consideramos que la cantidad de registros es grande, luego esto podrá evitar una reescritura de todo. se considera que se realiza una update de la taxonomias cada 3 meses agregando sólo las nuevas clasificaciones/especies que hayan surgido en ese tiempo, el update de especies existentes no se incluye en el merge ya que se considera en este escenario que los datos son fijos para cada especie ya dada de alta.

In [ ]:
bronze_dir = "datalake/bronze/ebird_api"
taxon_raw_dir = f"{bronze_dir}/taxonomia_last"

In [ ]:
upsert_data_as_delta(
    df_Taxon,
    taxon_raw_dir,
    "target.speciesCode = source.speciesCode"
    )

In [ ]:
dt = DeltaTable(taxon_raw_dir)
print(f"Cant de filas: {dt.to_pandas().shape[0]}")

Cant de filas: 17415


**Fin escenario inventado**

**EXTRACCION INCREMENTAL**


**Caso1
Endpoint "Historic observations on a date"**, url ("https://api.ebird.org/v2/data/obs/{{regionCode}}/historic/{{y}}/{{m}}/{{d}}") : es adecuado para una extracción incremental en la mayoría de los casos. Permite recuperar observaciones en una región específica en una fecha específica, ideal para actualizar una base de datos con nuevos registros de días seleccionados. Y también, con las nuevas observaciones de especies subidas **en el día actual**, ya que se actualizan estos datos varias veces en el día. Cada 30 a 60 minutos aproximadamente. Si bien este dato no está tan claro en el endpoint si comprobé que los datos se actualizan constantemente. También creo que podría utilizarse para una extracción full ya que, sirve para reconstruir datos históricos completos para períodos extensos pero su diseño está más orientado a actualizaciones incrementales.




Caso principal : **OBSERVACIONES HISTORICAS EN ARGENTINA (AR) durante la jornada presente** Se utiliza el Upsert sin particiones, ya que los datos que se obtienen son del día y se actualizan constantemente.

In [ ]:

fechas = parser["fechas"]
dates= fechas["dias"]


In [ ]:
import ast
# Convertir el string en una lista
dates = ast.literal_eval(dates)



In [ ]:
# Obtener datos correspondientes a las fechas
all_dates = []

for date in dates:
   url =  'https://api.ebird.org/v2/data/obs/' + lugar + '/historic/' + date
   json_data = get_data(url, params=params, headers=headers)
   if json_data:
     df = pd.DataFrame(json_data)
     all_dates.append(df)

# Concatenar todos los DataFrames en uno solo
df_Observ = pd.concat(all_dates, ignore_index=True)

# Mostrar la tabla final
print(df_Observ)

El codigo de respuesta es : 200
El codigo de respuesta es : 200
El codigo de respuesta es : 200
     speciesCode                  comName                 sciName      locId  \
0        eardov1               Eared Dove      Zenaida auriculata  L28219935   
1         hrshaw            Harris's Hawk    Parabuteo unicinctus  L22573219   
2         swahaw          Swainson's Hawk         Buteo swainsoni  L22573219   
3         grhowl         Great Horned Owl        Bubo virginianus  L22573219   
4         fuwduc   Fulvous Whistling-Duck     Dendrocygna bicolor  L21329515   
...          ...                      ...                     ...        ...   
1624      amgplo   American Golden-Plover      Pluvialis dominica    L844895   
1625      rudtur          Ruddy Turnstone      Arenaria interpres    L844895   
1626      bubsan  Buff-breasted Sandpiper  Calidris subruficollis    L844895   
1627      blkski            Black Skimmer          Rynchops niger    L844895   
1628     gubter2        

In [ ]:
df_Observ.head()

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory
0,eardov1,Eared Dove,Zenaida auriculata,L28219935,"350 Donnet, Esperanza, Santa Fe, AR (-31,447, ...",2023-12-10 23:23,1.0,-31.447323,-60.916328,True,True,True,S156118770,NaN
1,hrshaw,Harris's Hawk,Parabuteo unicinctus,L22573219,PN El Palmar--Sendero del Pastizal,2023-12-10 23:16,1.0,-31.888027,-58.241118,True,False,False,S156332511,NaN
2,swahaw,Swainson's Hawk,Buteo swainsoni,L22573219,PN El Palmar--Sendero del Pastizal,2023-12-10 23:16,20.0,-31.888027,-58.241118,True,False,False,S156332511,NaN
3,grhowl,Great Horned Owl,Bubo virginianus,L22573219,PN El Palmar--Sendero del Pastizal,2023-12-10 23:16,3.0,-31.888027,-58.241118,True,False,False,S156332511,NaN
4,fuwduc,Fulvous Whistling-Duck,Dendrocygna bicolor,L21329515,"Jardin 3 - 364 General Capdevila, Banfield, Pr...",2023-12-10 23:07,2.0,-34.742777,-58.381188,True,False,True,S156118320,NaN


In [ ]:
#almacenamiento
df_Observ["obsDt"] = pd.to_datetime(df_Observ["obsDt"], errors='coerce')
df_Observ["fecha"] = df_Observ.obsDt.dt.date
bronze_dir = "datalake/bronze/ebird_api"
observ_raw_dir = f"{bronze_dir}/ObservHist"

#ejecuta merge con insert / update particionando por fecha
upsert_data_as_delta(
    df_Observ,
    observ_raw_dir,
    "target.speciesCode = source.speciesCode AND target.fecha = source.fecha",
    partition_cols= ["fecha"]
    )

In [ ]:
dt = DeltaTable(observ_raw_dir)
print(f"Cant de filas: {dt.to_pandas().shape[0]}")

Cant de filas: 1629


In [ ]:
# Si se desea tener la info en formato CSV
Nombre = "ObsHis"
conv_to_csv(dt, Nombre)

ObsHis.csv


**TRANSFORMACIÓN:** Comenzamos la transformación sea usando el frame que ya tenemos en memoria o cargandolo desde deltaLake bronze.
En este caso realicé las siguientes operaciones:
 Eliminación o reemplazo de nulos, Conversión de tipos de datos de columnas,Renombrar columnas, eliminación de columna entera y agregaciones por medio de GROUP BY y funciones count. No obtuve una reducción de la memoria dada la poca cantidad de registros de la tabla (esto se pudo hacer en el caso anterior). En este caso, se realiza además un join de ambas tablas almacenadas en silver (taxonomía y Observaciones para complementar la info de ambas y se almacena en capa oro.

In [ ]:
# cargandolo desde deltaLake bronze si es necesario y no lo tenemos ya en memoria
dt = DeltaTable(observ_raw_dir)

In [ ]:
df = dt.to_pandas()




In [ ]:
# "memory_usage"
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1629 entries, 0 to 1628
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   speciesCode      1629 non-null   object        
 1   comName          1629 non-null   object        
 2   sciName          1629 non-null   object        
 3   locId            1629 non-null   object        
 4   locName          1629 non-null   object        
 5   obsDt            1629 non-null   datetime64[us]
 6   howMany          1579 non-null   float64       
 7   lat              1629 non-null   float64       
 8   lng              1629 non-null   float64       
 9   obsValid         1629 non-null   bool          
 10  obsReviewed      1629 non-null   bool          
 11  locationPrivate  1629 non-null   bool          
 12  subId            1629 non-null   object        
 13  exoticCategory   33 non-null     object        
 14  fecha            1629 non-null   object 

In [ ]:
# Contamos la cantidad de valores nulos en el campo clave
df["speciesCode"].isnull().sum()

0

In [ ]:
df.head()

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,fecha
0,batman1,Band-tailed Manakin,Pipra fasciicauda,L2349765,Yacutinga Lodge,2023-12-11 15:58:00,1.0,-25.585753,-54.06871,True,False,False,S156162773,None,2023-12-11
1,rucspi1,Rufous-capped Spinetail,Synallaxis ruficapilla,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,2.0,-25.585753,-54.06871,True,False,False,S156130089,None,2023-12-11
2,swtman1,Swallow-tailed Manakin,Chiroxiphia caudata,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,1.0,-25.585753,-54.06871,True,False,False,S156130089,None,2023-12-11
3,eaptyr1,Eared Pygmy-Tyrant,Myiornis auricularis,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,1.0,-25.585753,-54.06871,True,False,False,S156130089,None,2023-12-11
4,smbtin1,Small-billed Tinamou,Crypturellus parvirostris,L2349765,Yacutinga Lodge,2023-12-11 06:17:00,2.0,-25.585753,-54.06871,True,False,False,S156135844,None,2023-12-11


In [ ]:
df["howMany"].isnull().sum()

50

In [ ]:
#reemplazo con valor 0 los nulos para poder cambiar a entero pero no se borraran las filas con valor 0 porque son 50 y tienen info importante.
df['howMany'] = df['howMany'].fillna(0)

In [ ]:
df["howMany"] = df["howMany"].astype("int8")

In [ ]:
df.head()

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,fecha
0,batman1,Band-tailed Manakin,Pipra fasciicauda,L2349765,Yacutinga Lodge,2023-12-11 15:58:00,1,-25.585753,-54.06871,True,False,False,S156162773,None,2023-12-11
1,rucspi1,Rufous-capped Spinetail,Synallaxis ruficapilla,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,2,-25.585753,-54.06871,True,False,False,S156130089,None,2023-12-11
2,swtman1,Swallow-tailed Manakin,Chiroxiphia caudata,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,1,-25.585753,-54.06871,True,False,False,S156130089,None,2023-12-11
3,eaptyr1,Eared Pygmy-Tyrant,Myiornis auricularis,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,1,-25.585753,-54.06871,True,False,False,S156130089,None,2023-12-11
4,smbtin1,Small-billed Tinamou,Crypturellus parvirostris,L2349765,Yacutinga Lodge,2023-12-11 06:17:00,2,-25.585753,-54.06871,True,False,False,S156135844,None,2023-12-11


In [ ]:
# Miremos al final donde dice "memory_usage"
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1629 entries, 0 to 1628
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   speciesCode      1629 non-null   object        
 1   comName          1629 non-null   object        
 2   sciName          1629 non-null   object        
 3   locId            1629 non-null   object        
 4   locName          1629 non-null   object        
 5   obsDt            1629 non-null   datetime64[us]
 6   howMany          1629 non-null   int8          
 7   lat              1629 non-null   float64       
 8   lng              1629 non-null   float64       
 9   obsValid         1629 non-null   bool          
 10  obsReviewed      1629 non-null   bool          
 11  locationPrivate  1629 non-null   bool          
 12  subId            1629 non-null   object        
 13  exoticCategory   33 non-null     object        
 14  fecha            1629 non-null   object 

In [ ]:
df

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,fecha
0,patmoc1,Patagonian Mockingbird,Mimus patagonicus,L28143881,"Auto selected -39.64478, -64.43221",2023-12-01 16:02:00,1,-39.644781,-64.432210,True,False,True,S155557950,None,2023-12-01
1,trsowl,Tropical Screech-Owl,Megascops choliba,L23194663,"Vía sin nombre, Corrientes, AR (-27.612, -56.411)",2023-12-01 23:07:00,1,-27.611788,-56.410798,True,False,True,S155560805,None,2023-12-01
2,bufowl1,Buff-fronted Owl,Aegolius harrisii,L17905826,Cuesta del Totoral,2023-12-01 22:39:00,2,-28.123429,-65.633989,True,False,False,S155565017,None,2023-12-01
3,bawnig1,Band-winged Nightjar,Systellura longirostris,L9647779,Chorrillo del Salto,2023-12-01 21:40:00,1,-49.295529,-72.907514,True,False,False,S155560672,None,2023-12-01
4,soulap1,Southern Lapwing,Vanellus chilensis,L28144457,Rosario,2023-12-01 21:10:00,1,-32.901942,-60.770302,True,False,True,S155561652,None,2023-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,stisan,Stilt Sandpiper,Calidris himantopus,L2507409,Ceibas--humedales,2023-12-11 06:30:00,9,-33.502181,-58.671424,True,True,False,S156149714,None,2023-12-11
1625,whnxen1,White-naped Xenopsaris,Xenopsaris albinucha,L2507409,Ceibas--humedales,2023-12-11 06:30:00,1,-33.502181,-58.671424,True,False,False,S156149714,None,2023-12-11
1626,banswa,Bank Swallow,Riparia riparia,L2507409,Ceibas--humedales,2023-12-11 06:30:00,2,-33.502181,-58.671424,True,False,False,S156149714,None,2023-12-11
1627,cliswa,Cliff Swallow,Petrochelidon pyrrhonota,L2507409,Ceibas--humedales,2023-12-11 06:30:00,1,-33.502181,-58.671424,True,False,False,S156149714,None,2023-12-11


Se decide borrar la columna obsReviewed porque es un dato que no influye en la validez de la observación  (colummna obsValid)

In [ ]:
df = df = df.drop("obsReviewed", axis=1)

In [ ]:
df.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1629 entries, 0 to 1628
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   speciesCode      1629 non-null   object        
 1   comName          1629 non-null   object        
 2   sciName          1629 non-null   object        
 3   locId            1629 non-null   object        
 4   locName          1629 non-null   object        
 5   obsDt            1629 non-null   datetime64[us]
 6   howMany          1629 non-null   int8          
 7   lat              1629 non-null   float64       
 8   lng              1629 non-null   float64       
 9   obsValid         1629 non-null   bool          
 10  locationPrivate  1629 non-null   bool          
 11  subId            1629 non-null   object        
 12  exoticCategory   33 non-null     object        
 13  fecha            1629 non-null   object        
dtypes: bool(2), datetime64[us](1), float64(2

**Finalmente se logra una reducción de memoria de 16.5 MB a memory usage: 13.4 MB**

In [ ]:
df


,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,locationPrivate,subId,exoticCategory,fecha
0,batman1,Band-tailed Manakin,Pipra fasciicauda,L2349765,Yacutinga Lodge,2023-12-11 15:58:00,1,-25.585753,-54.068710,True,False,S156162773,None,2023-12-11
1,rucspi1,Rufous-capped Spinetail,Synallaxis ruficapilla,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,2,-25.585753,-54.068710,True,False,S156130089,None,2023-12-11
2,swtman1,Swallow-tailed Manakin,Chiroxiphia caudata,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,1,-25.585753,-54.068710,True,False,S156130089,None,2023-12-11
3,eaptyr1,Eared Pygmy-Tyrant,Myiornis auricularis,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,1,-25.585753,-54.068710,True,False,S156130089,None,2023-12-11
4,smbtin1,Small-billed Tinamou,Crypturellus parvirostris,L2349765,Yacutinga Lodge,2023-12-11 06:17:00,2,-25.585753,-54.068710,True,False,S156135844,None,2023-12-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,amgplo,American Golden-Plover,Pluvialis dominica,L844895,Gral. Lavalle,2023-12-01 00:00:00,12,-36.401372,-56.951113,True,False,S156027685,None,2023-12-01
1625,rudtur,Ruddy Turnstone,Arenaria interpres,L844895,Gral. Lavalle,2023-12-01 00:00:00,1,-36.401372,-56.951113,True,False,S156027685,None,2023-12-01
1626,bubsan,Buff-breasted Sandpiper,Calidris subruficollis,L844895,Gral. Lavalle,2023-12-01 00:00:00,8,-36.401372,-56.951113,True,False,S156027685,None,2023-12-01
1627,blkski,Black Skimmer,Rynchops niger,L844895,Gral. Lavalle,2023-12-01 00:00:00,50,-36.401372,-56.951113,True,False,S156027685,None,2023-12-01


**Agrupamos por fecha/especies  y obtenemos la cantidad de observaciones de especies, de cada una de las especies observadas en las fechas trabajadas**


In [ ]:
df.groupby(['fecha','speciesCode']
           ).agg(
               {
                'howMany': 'mean'
                 }
          ).rename(
        columns={
            'howMany': 'Cant de obs por especie en la fecha'
            }

          )


Cant de obs por especie en la fecha
fecha      speciesCode                                     
2023-12-01 absfin1                                      2.0
           amakin1                                      2.0
           amekes                                       1.0
           ameoys                                      18.0
           amgplo                                      12.0
...                                                     ...
2023-12-11 yermar1                                      1.0
           yewbla2                                      3.0
           ysbfin1                                      1.0
           yunman1                                      1.0
           zimtap1                                      1.0

[1629 rows x 1 columns]

In [ ]:
#almacenamos en capa silver
df["obsDt"] = pd.to_datetime(df["obsDt"], errors='coerce')
df["fecha"] = df.obsDt.dt.date
bronze_dir = "datalake/silver/ebird_api"
observ_raw_dir = f"{bronze_dir}/ObservHist"

#ejecuta merge con insert / update particionando por fecha
upsert_data_as_delta(
    df,
    observ_raw_dir,
    "target.speciesCode = source.speciesCode AND target.fecha = source.fecha",
    partition_cols= ["fecha"]
    )

In [ ]:
#A modo de chequeo, leemos los datos guardados y contamos la cantidad de fila
dt = DeltaTable(observ_raw_dir)
print(f"Cant de filas: {dt.to_pandas().shape[0]}")

Cant de filas: 1629


**Almacenamos agregación en capa Gold**

In [ ]:
#Aqui tenemos df las observaciones, leemos la taxonomia y hacemos un join para completar en la nueva tabla los datos de la especie
dt = DeltaTable(taxon_raw_dir)
dx = dt.to_pandas()
#dx_limited = dx['comName'].tolist()
dx_limited = dx[['speciesCode', 'sciName','comName','order','familyComName','familySciName']]

merged_df = pd.merge(df, dx_limited, on='speciesCode', how='inner')

In [ ]:
merged_df.columns.values.tolist()

['speciesCode',
 'comName_x',
 'sciName_x',
 'locId',
 'locName',
 'obsDt',
 'howMany',
 'lat',
 'lng',
 'obsValid',
 'locationPrivate',
 'subId',
 'exoticCategory',
 'fecha',
 'sciName_y',
 'comName_y',
 'order',
 'familyComName',
 'familySciName']

In [ ]:
merged_df


,speciesCode,comName_x,sciName_x,locId,locName,obsDt,howMany,lat,lng,obsValid,locationPrivate,subId,exoticCategory,fecha,sciName_y,comName_y,order,familyComName,familySciName
0,batman1,Band-tailed Manakin,Pipra fasciicauda,L2349765,Yacutinga Lodge,2023-12-11 15:58:00,1,-25.585753,-54.068710,True,False,S156162773,None,2023-12-11,Pipra fasciicauda,Band-tailed Manakin,Passeriformes,Manakins,Pipridae
1,rucspi1,Rufous-capped Spinetail,Synallaxis ruficapilla,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,2,-25.585753,-54.068710,True,False,S156130089,None,2023-12-11,Synallaxis ruficapilla,Rufous-capped Spinetail,Passeriformes,Ovenbirds and Woodcreepers,Furnariidae
2,swtman1,Swallow-tailed Manakin,Chiroxiphia caudata,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,1,-25.585753,-54.068710,True,False,S156130089,None,2023-12-11,Chiroxiphia caudata,Swallow-tailed Manakin,Passeriformes,Manakins,Pipridae
3,eaptyr1,Eared Pygmy-Tyrant,Myiornis auricularis,L2349765,Yacutinga Lodge,2023-12-11 06:18:00,1,-25.585753,-54.068710,True,False,S156130089,None,2023-12-11,Myiornis auricularis,Eared Pygmy-Tyrant,Passeriformes,Tyrant Flycatchers,Tyrannidae
4,smbtin1,Small-billed Tinamou,Crypturellus parvirostris,L2349765,Yacutinga Lodge,2023-12-11 06:17:00,2,-25.585753,-54.068710,True,False,S156135844,None,2023-12-11,Crypturellus parvirostris,Small-billed Tinamou,Tinamiformes,Tinamous,Tinamidae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,amgplo,American Golden-Plover,Pluvialis dominica,L844895,Gral. Lavalle,2023-12-01 00:00:00,12,-36.401372,-56.951113,True,False,S156027685,None,2023-12-01,Pluvialis dominica,American Golden-Plover,Charadriiformes,Plovers and Lapwings,Charadriidae
1625,rudtur,Ruddy Turnstone,Arenaria interpres,L844895,Gral. Lavalle,2023-12-01 00:00:00,1,-36.401372,-56.951113,True,False,S156027685,None,2023-12-01,Arenaria interpres,Ruddy Turnstone,Charadriiformes,Sandpipers and Allies,Scolopacidae
1626,bubsan,Buff-breasted Sandpiper,Calidris subruficollis,L844895,Gral. Lavalle,2023-12-01 00:00:00,8,-36.401372,-56.951113,True,False,S156027685,None,2023-12-01,Calidris subruficollis,Buff-breasted Sandpiper,Charadriiformes,Sandpipers and Allies,Scolopacidae
1627,blkski,Black Skimmer,Rynchops niger,L844895,Gral. Lavalle,2023-12-01 00:00:00,50,-36.401372,-56.951113,True,False,S156027685,None,2023-12-01,Rynchops niger,Black Skimmer,Charadriiformes,"Gulls, Terns, and Skimmers",Laridae


In [ ]:
#almacenamos en capa Gold
merged_df["obsDt"] = pd.to_datetime(merged_df["obsDt"], errors='coerce')
merged_df["fecha"] = merged_df.obsDt.dt.date
bronze_dir = "datalake/gold/ebird_api"
observ_raw_dir = f"{bronze_dir}/ObservHist"


#ejecuta merge con insert / update particionando por fecha
upsert_data_as_delta(
    merged_df,
    observ_raw_dir,
    "target.speciesCode = source.speciesCode AND target.fecha = source.fecha",
    partition_cols= ["fecha"]
    )

**Caso2 Endpoint :Recent observations in a region** (https://api.ebird.org/v2/data/obs/{{regionCode}}/recent) 2do caso para Extr. Incremental: Por default me trae las observaciones de una región/país en los últimos 14 días. Se utilizará Merge (insert/update y particiones),se almacenarán los datos particionando por las 14 fechas. Al volver a correr se actualiza la partición del día corriente solamente. Al día siguiente debería actualizar lo que queda del día anterior y crear una nueva partición y actualizará sobre ésta hasta la 1era ejecución del día siguiente.


In [ ]:
url =  'https://api.ebird.org/v2/data/obs/' + lugar + '/recent'


In [ ]:
params = {}
json_data = get_data(url, params=params, headers=headers)
df_Observ = build_table(json_data)


El codigo de respuesta es : 200


In [ ]:
#print(df_Observ)
df_Observ.sort_values("obsDt", ascending=True)

    speciesCode                   comName                    sciName  \
0       litnig1           Little Nightjar          Setopagis parvula   
1       chicar1         Chimango Caracara          Daptrius chimango   
2       ausbla1         Austral Blackbird            Curaeus curaeus   
3       rufhor2            Rufous Hornero            Furnarius rufus   
4       redsho1              Red Shoveler           Spatula platalea   
..          ...                       ...                        ...   
844     thbsal1     Thick-billed Saltator        Saltator maxillosus   
845     baceag2  Black-and-chestnut Eagle          Spizaetus isidori   
846     goreup1    Golden-rumped Euphonia  Chlorophonia cyanocephala   
847     gretyr1       Greenish Tyrannulet      Phyllomyias virescens   
848     pefpar1    Peach-fronted Parakeet           Eupsittula aurea   

         locId                                       locName  \
0    L21882718                   Solárium Suizo, Agua de Oro   
1    L2

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory
848,pefpar1,Peach-fronted Parakeet,Eupsittula aurea,L12467820,"Formosa, AR (-26,232, -58,168)",2024-10-30 06:25,1.0,-26.232179,-58.167837,True,False,True,S200781737,NaN
847,gretyr1,Greenish Tyrannulet,Phyllomyias virescens,L495969,PP Urugua-í--Seccional Uruzú,2024-10-30 06:33,2.0,-25.857317,-54.168783,True,False,False,S200822206,NaN
846,goreup1,Golden-rumped Euphonia,Chlorophonia cyanocephala,L2936655,Reserva Natural Privada Eco-Portal de Piedra,2024-10-30 10:09,1.0,-24.098342,-64.400300,True,False,False,S200785636,NaN
845,baceag2,Black-and-chestnut Eagle,Spizaetus isidori,L3843279,Humedales Dique Los Molinos--Ladera NE,2024-10-30 11:00,1.0,-24.155596,-65.368180,True,False,False,S200848048,NaN
844,thbsal1,Thick-billed Saltator,Saltator maxillosus,L859331,PP Caá Yarí,2024-10-30 18:02,1.0,-26.868411,-54.217100,True,True,False,S200850822,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,redsho1,Red Shoveler,Spatula platalea,L3314952,Trelew--Laguna Chiquichano,2024-11-12 20:02,300.0,-43.248079,-65.299301,True,False,False,S202306499,NaN
3,rufhor2,Rufous Hornero,Furnarius rufus,L3314952,Trelew--Laguna Chiquichano,2024-11-12 20:02,1.0,-43.248079,-65.299301,True,False,False,S202306499,NaN
2,ausbla1,Austral Blackbird,Curaeus curaeus,L21722535,Ea La Catalana--Casco,2024-11-12 20:06,2.0,-54.175069,-67.278693,True,False,False,S202306253,NaN
1,chicar1,Chimango Caracara,Daptrius chimango,L21722535,Ea La Catalana--Casco,2024-11-12 20:06,1.0,-54.175069,-67.278693,True,False,False,S202306253,NaN


In [ ]:
from datetime import datetime, timedelta
df_Observ["obsDt"] = pd.to_datetime(df_Observ["obsDt"], errors='coerce')
df_Observ["fecha"] = df_Observ.obsDt.dt.date
#no se trabaja con la hora porque se actualizan algunos campos en el registro correspondiente a la especie la especie con el total de observaciones que van en el día
# también los campos localidad y fecha/hora ya que se actualiza con los datos de la última observación en
#el país del correspondiente cod de especie

In [ ]:
fecha_actual = datetime.now().strftime("%Y_%m_%d")
bronze_dir = "datalake/bronze/ebird_api"
observ_raw_dir = f"{bronze_dir}/ObsRec-{fecha_actual}"
upsert_data_as_delta(
    df_Observ,
    observ_raw_dir,
    "target.speciesCode = source.speciesCode AND target.fecha = source.fecha",
     partition_cols= ["fecha"]
    )

In [ ]:
dt = DeltaTable(observ_raw_dir)
print(f"Cant de filas: {dt.to_pandas().shape[0]}")

Cant de filas: 849


In [ ]:
Nombre = "ObsRecAl"
conv_to_csv(dt, Nombre)

ObsRecAl13-11-2024.csv
